In [1]:
using Pkg; #Pkg.add("PyCall"); Pkg.add("Conda"); Pkg.add("DataFrames"); Pkg.add("XLSX")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed ZipFile ─ v0.8.3
 Installed XLSX ──── v0.5.5
  Updating `~/.julia/environments/v1.2/Project.toml`
  [fdbf4ff8] + XLSX v0.5.5
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [fdbf4ff8] + XLSX v0.5.5
  [a5390f91] + ZipFile v0.8.3
  Building ZipFile → `~/.julia/packages/ZipFile/oD4uG/deps/build.log`


In [2]:
using Conda
Conda.add("openpyxl")

┌ Info: Running `conda install -y openpyxl` in root environment
└ @ Conda /home/nakada/.julia/packages/Conda/kLXeC/src/Conda.jl:112


Solving environment: ...working... done

## Package Plan ##

  environment location: /home/nakada/.julia/conda/3

  added / updated specs:
    - openpyxl


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    et_xmlfile-1.0.1           |           py37_0          21 KB
    jdcal-1.4.1                |             py_0          11 KB
    openpyxl-2.6.2             |             py_0         157 KB
    ------------------------------------------------------------
                                           Total:         189 KB

The following NEW packages will be INSTALLED:

  et_xmlfile         pkgs/main/linux-64::et_xmlfile-1.0.1-py37_0
  jdcal              pkgs/main/noarch::jdcal-1.4.1-py_0
  openpyxl           pkgs/main/noarch::openpyxl-2.6.2-py_0



openpyxl-2.6.2       | 157 KB    | ########## | 100% 
jdcal-1.4.1          | 11 KB     | ########## | 100% 
et_xmlfile-1.0.1     | 21 KB     | #

In [3]:
using PyCall
using Dates
using Random
xl = pyimport("openpyxl")

PyObject <module 'openpyxl' from '/home/nakada/.julia/conda/3/lib/python3.7/site-packages/openpyxl/__init__.py'>

In [4]:
wb = xl.Workbook();
ws = wb.active

PyObject <Worksheet "Sheet">

In [5]:
ws.cell(1, 1, "Data generated on:")

PyObject <Cell 'Sheet'.A1>

In [6]:
ws.cell(2, 1, Dates.now())

PyObject <Cell 'Sheet'.A2>

In [7]:
Random.seed!(0)
dat = rand(3, 5)
for i in 1:size(dat)[1]
    ws.append((dat[i, :]..., ))
end

In [8]:
wb.save("sample1.xlsx")

In [9]:
wb = xl.load_workbook(filename = "sample1.xlsx")
ws = wb.active

PyObject <Worksheet "Sheet">

In [10]:
println(ws.cell(1, 1).value, "\n", ws.cell(2, 1).value)

Data generated on:
2019-09-01T08:49:13.819


In [11]:
typeof(ws.cell(2, 1).value)

DateTime

In [12]:
ws.cell(20, 221).value == nothing

true

In [13]:
using Printf

wb = xl.load_workbook(filename = "sample1.xlsx")
ws = wb.active

for row in ws.rows
    for cell in row
        print("|")
        if typeof(cell.value) <: Number
            @printf("%.3f", cell.value)
        else
            show(cell.value)
        end
    end
    println("|")
end

|"Data generated on:"|nothing|nothing|nothing|nothing|
|2019-09-01T08:49:13.819|nothing|nothing|nothing|nothing|
|0.824|0.177|0.042|0.973|0.260|
|0.910|0.279|0.068|0.586|0.910|
|0.165|0.203|0.362|0.539|0.167|


In [14]:
wb.close()

-----

In [15]:
using XLSX

┌ Info: Precompiling XLSX [fdbf4ff8-1666-58a4-91e7-1b58723a45e0]
└ @ Base loading.jl:1242


In [16]:
XLSX.openxlsx("sample2.xlsx", mode="w") do xf
    sheet = xf[1]
    XLSX.rename!(sheet, "SheetName")
    sheet["A1"] = "Data generated on:"
    sheet["A2"] = Dates.now()
    dat = rand(3, 5)
    for row in 1:3
        for col in 1:5
            XLSX.setdata!(sheet, XLSX.CellRef(2+row, col), rand())
        end 
    end
end

In [17]:
wb = XLSX.readxlsx("sample2.xlsx")

XLSXFile("sample2.xlsx") containing 1 Worksheets
            sheetname size          range        
-------------------------------------------------
            SheetName 5x5           A1:E5        


In [18]:
sheetname = XLSX.sheetnames(wb)[1]

"SheetName"

In [19]:
ws = wb[sheetname]

5×5 XLSX.Worksheet: ["SheetName"](A1:E5)

In [20]:
dim = ws.dimension

A1:E5

In [21]:
ws[dim]

5×5 Array{Any,2}:
  "Data generated on:"   missing   missing    missing   missing 
  2019-09-01T08:50:00    missing   missing    missing   missing 
 0.838118               0.914712  0.300075   0.72285   0.119653 
 0.76707                0.801924  0.0353445  0.484661  0.899199 
 0.951691               0.801119  0.124323   0.114269  0.0795545

In [22]:
XLSX.close(wb)

In [23]:
using DataFrames
df1 = DataFrame(a=[1, 2, 3], b=[4, 5, 6]);
df2 = DataFrame(x1=[1, 2, 3], x2=["A", "B", "C"]);

In [24]:
XLSX.writetable("sample3.xlsx",
    SheetName1=( DataFrames.eachcol(df1), DataFrames.names(df1) ),
    SheetName2=( DataFrames.eachcol(df2), DataFrames.names(df2) ));


AssertionError: AssertionError: sample3.xlsx already exists.